In [1]:
import numpy as p 
import pandas as pd

In [2]:
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')

In [3]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0.0,36.0,3.0,2.0,2.0,1.0,850.0,0.0,0.0,0.0,1.0,1.0,0.82
1,0.0,95.0,2.0,2.0,2.0,1.0,1226.0,1.0,0.0,0.0,1.0,2.0,0.95
2,0.0,103.0,2.0,2.0,1.0,1.0,1000.0,0.0,0.0,0.0,1.0,0.0,0.32
3,0.0,99.0,3.0,4.0,4.0,3.0,1615.0,1.0,0.0,1.0,0.0,2.0,1.60
4,0.0,5.0,2.0,2.0,1.0,3.0,582.0,0.0,1.0,0.0,0.0,2.0,0.48


In [24]:
x = df.drop(columns=['price'])
y =df['price'] 

In [25]:
from sklearn.model_selection import KFold,cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [26]:
column_to_encode = ['sector','balcony', 'furnishing_type','luxury_category','floor_category','agePossession']

In [27]:
y_transform = np.log1p(y)

<IPython.core.display.Javascript object>

In [31]:
# creating a column transformer for preprocessing
preprocessor =ColumnTransformer(
transformers=[
    ('num', StandardScaler(),['property_type','bedRoom','bathroom','built_up_area','servant room','store room'] ),
    ('cat', OneHotEncoder(drop='first'),column_to_encode)
],
 remainder= 'passthrough'
)

In [32]:
# creating a pipeline
pipeline = Pipeline([
    ('preprocessor',preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [33]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)  # Change 'kflod' to 'kfold'
scores = cross_val_score(pipeline, x, y_transform, cv=kfold, scoring='r2')

In [34]:
scores.mean()

0.8845360715052786

In [35]:
scores.std()

0.014784881452420021

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
x_train,x_test,y_train,y_test = train_test_split(x,y_transform,test_size=0.2,random_state=42)

In [41]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category',
                                                   'agePossession'])])),
                ('regressor', SVR())])

In [42]:
y_pred =pipeline.predict(x_test)

In [44]:
y_pred= np.expm1(y_pred)

<IPython.core.display.Javascript object>

In [45]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

<IPython.core.display.Javascript object>

0.5324591082613233